# Gaussian Models

We evaluate calibration tests empirically similar to the synthetic experiments in the paper [Calibration tests beyond classification](https://openreview.net/forum?id=-bxf89v3Nx) by Widmann, Lindsten, and Zachariah.

For fixed target dimension $d \geq 1$, we generate datasets corresponding to (un)calibrated models using the following procedure:
1. Sample iid $c^i \sim \mathcal{N}(0, 1)$
1. Define Gaussian predictions $g^i = \mathcal{N}(c^i \mathbf{1}_d, \sigma^2 \mathbf{I}_d)$ ($\mathbf{I}_d \in \mathbb{R}^{d \times d}$ denotes the identity matrix of size $d \times d$)
2. Define Gaussian predictions $\hat{g}^i = g^i + \delta \mathbf{1}_d = \mathcal{N}((c^i + \delta) \mathbf{1}_d, \sigma^2 \mathbf{I}_d)$ for some fixed $\delta \in \mathbb{R} \setminus \{0\}$
2. Draw an observation $y^i \sim g^i$ for each prediction $g(X)^i$

Note that calibration analysis is only based on the observations and corresponding predicted distributions, and hence we do neither specify features nor explicit models and instead only define the distributions of the observations and predictions.
One could view $c$ as normally distributed features, $g$ as a Gaussian model of the form $x \mapsto \mathcal{N}(x \mathbf{1}_d, \sigma^2 \mathbf{I}_d)$, and $\hat{g}$ as a Gaussian model $x \mapsto \mathcal{N}(x \mathbf{1}_d, \sigma^2 \mathbf{I}_d)$.
However, this is only one possible interpretation and many other features and models also correspond to the setting above.

Then $\{g^i, y^i\}_i$ defines a set of iid samples of the predictions of a calibrated model and corresponding observations:
By design the sampling procedure ensures that
$$
p(y^i \in \cdot \mid g^i) = g^i.
$$
On the other hand, the algorithm above yields
$$
p(y^i \in \cdot \mid \hat{g}^i) = p(y^i \in \cdot \mid g^i = \hat{g}^i - \delta \mathbf{1}_d) = \hat{g}^i - \delta \mathbf{1}_d
$$
and hence almost surely
$$
p(y^i \in \cdot \mid \hat{g}^i) \neq \hat{g}^i,
$$
i.e., $\{\hat{g}^i, y^i\}_i$ corresponds to a set of iid samples of an uncalibrated model for the same observations.


The main difference to the setup in the paper [Calibration tests beyond classification](https://openreview.net/forum?id=-bxf89v3Nx) is that there the authors used a fixed set of predictions with different sets of observations whereas here we use a fixed set of targets with different sets of predictions.
The motivation for our choice here is that in practice usually one wants to evaluate different models for a given set observations.

## Kernels

We compare calibration tests with the the kernel calibration-conditional Stein discrepancy (KCCSD) and the kernel calibration error (KCE), using different kernels $k_p$ and $k_y$ on the space of predictions and targets, respectively.

In this example, the target space is $\mathbb{R}^d$ and hence for $k_y$ we can choose standard kernels.
We consider
1. the Gaussian kernel $k_y(x, y) = \exp{(- \|x - y\|_2^2 / (2 \sigma_y^2))}$ with length scale $\sigma_y > 0$, and
2. the inverse multiquadric kernel $k_y(x, y) = {(1 + \|x - y\|_2^2 / \sigma_y^2)}^{-1/2}$ with length scale $\sigma_y > 0$.

The space of predictions is a subspace of isotropic Gaussian distributions, namely $\mathcal{G} := \left\{\mathcal{N}(x \mathbf{1}_d, \sigma^2 \mathbf{I}_d) \colon x \in \mathbb{R} \right\}$.
Similar to our choices for $k_y$ above, we study
1. the Gaussian kernel $k_p(P, P') = \exp{(- \|P - P'\|^2 / (2 \sigma_p^2))}$ with length scale $\sigma_p > 0$, and
2. the inverse multiquadric kernel $k_p(P, P') = {(1 + \|P - P'\|^2 / \sigma_p^2)}^{-1/2}$ with length scale $\sigma_p > 0$.

However, for $k_p$ we do use dedicated metrics on $\mathcal{G}$ that are available in closed form:
the 2-Wasserstein distance with respect to the Euclidean distance and
the maximum-mean discrepancy (MMD) with respect to the Gaussian kernel $k(x, y) = \exp{(- \|x - y\|_2^2 / (2 \sigma_k^2))}$ with length scale $\sigma_k > 0$.
The 2-Wasserstein distance $W_2(P, P')$ between isotropic Gaussian distributions $P, P' \in \mathcal{G}$ is given by
$$
W_2(P, P) = \left\|\mathbb{E}_{X \sim P} X - \mathbb{E}_{X \sim P'} X\right\|_2.
$$
On the other hand, the MMD with respect to the Gaussian kernel is given by
$$
\operatorname{MMD}_k(P, P')^2 = \frac{2^d (1 - \exp{(- \|\mathbb{E}_{X \sim P} X - \mathbb{E}_{X \sim P'} X)\|_2^2 / (2 (2\sigma^2 + \sigma_k^2))})}{(1 + 2 (\sigma / \sigma_k)^2)^{d / 2}}.
$$

Additionally, for $k_p$ we study the measure-transport based kernel with the MMD with a Gaussian kernel.

For all kernels, length scales are set with the median heuristic to $\operatorname{median} \{\|x_i - x_j\| \colon 1 \leq i, j \leq n\}$, i.e., to the median of the pairwise distances.
Since both $W_2$ and $\operatorname{MMD}_k(P, P')$ only depend on the *difference* of the means of $P$ and $P'$,
in particular we have $W_2(P + \delta \mathbf{1}_d, P' + \delta \mathbf{1}_d) = W_2(P, P')$ and $\operatorname{MMD}_k(P + \delta \mathbf{1}_d, P' + \delta \mathbf{1}_d) = \operatorname{MMD}_k(P, P')$
which implies that the median heuristic yields the same length scales for $\{g^i\}_{i=1}^n$ and $\{\hat{g}^i\}_{i=1}^n$.

For some combinations of kernels, the test statistic of the KCCSD and/or the KCE are available in closed form.
The cases where we have to resort to sampling are:
- For both KCCSD and KCE: Use of the measure-transport based kernel as it has to be approximated with samples from the base measure.
- For the KCE only: Use of the inverse multiquadric kernel as its expectation with respect to its argument(s) following a Gaussian distribution is not available in closed form.

All other cases can be evaluated analytically.